In [73]:
require(reshape)
require(repr)
require(ggplot2)
require(dplyr)
require(survival)
require(zoo)
require(caret)

In [74]:
df <- read.csv('data/total_logistic.csv', header=TRUE, sep=",") %>% 
    filter(hospitalid==420 | hospitalid==443) %>%
    select(hospitalid, actualiculos, mortality2, age, GCS2, temperature2, ph2, pco22, bun2, bilirubin2, fio22) %>% 
    na.omit()

In [75]:
H1<- df %>% 
    filter(hospitalid == 420) %>%
    filter(actualiculos <= 50)
H2 <- df %>% 
    filter(hospitalid == 443) %>% 
    filter(actualiculos <= 50)
save(H1, H2, file='data/H_central.RData')

In [76]:
load('data/H_central.RData')

In [77]:
col.list <- H1 %>% names() 
col.list

[1] "hospitalid"   "actualiculos" "mortality2"   "age"          "GCS2"        
 [6] "temperature2" "ph2"          "pco22"        "bun2"         "bilirubin2"  
[11] "fio22"

In [78]:
# change the name of the variables to time, Y, and x1..xp

namer.function <- function(local.data, time, Y)
{
    df <- rename(local.data, "time"=time, "Y"=Y)
    df.Y <- subset(df, select = c("time","Y"))
    df.X <- subset(df, select = !(names(df) %in% c("time","Y")))
    
    # rename X's
    col.len <- df.X %>% names %>% length
    
    for(i in 1:col.len){
            names(df.X)[i] <- paste0('x',i)
        }
    changed.data <- cbind(df.Y, df.X)
    return(changed.data)
}

In [79]:
H1.changed <- namer.function(H1,'actualiculos','mortality2')
H2.changed <- namer.function(H2, 'actualiculos', 'mortality2')

In [80]:
H1.changed <- H1.changed %>% as_tibble()
H2.changed <- H2.changed %>% as_tibble()

In [81]:
Generate.Z1.Z2 <- function(local.data, predictor, p, m)
{   
    z1.list <- list()
    z2.list <- list()

    split <- function(data, y ,p)
    {
        z1.idx <- caret::createDataPartition(y = data[[y]], p=p, list=FALSE)
        z1<- data[z1.idx,]
        z2<- data[-z1.idx,]
        return(list(z1,z2))
    }

    for(i in 1:m){
        split(local.data, predictor, p)
        result <- split(local.data, predictor, p)

        z1.list <- append(z1.list, result[1])
        z2.list <- append(z2.list, result[2])
        if(i == m) break
        i = i + 1
    }

    return(list(z1.list, z2.list))
}

# 설명
# z1.list 와 z2.list 가 순서대로 m개씩 묶여있다. 

In [82]:
Z1.Z2.H1.100 <- Generate.Z1.Z2(H1.changed, 'Y', 0.7, 100)
Z1.Z2.H2.100 <- Generate.Z1.Z2(H2.changed, 'Y', 0.7, 100)

In [83]:
save(Z1.Z2.H1.100, file = 'data/z1z2H1.Rdata')
save(Z1.Z2.H2.100, file = 'data/z1z2H2.Rdata')

In [84]:
Z1.Z2.H1.100[[1]] %>% length

[1] 100

In [117]:
Z1.Z2.H1.100[[1]][1]["Y"]

[[1]]
NULL

In [133]:
# 모델 만들기

partyNameH1 <- "H1"
partyNameH2 <- "H2"


make.model <- function(z1z2list, party.name)
{
    m <- z1z2list[[1]] %>% length
    
    # 모델을 만든다. z1z2의 리스트에 있는 z1의 1~m개의 데이터를 바탕으로
    # 모델을 만든다.
    
    for(i in 1:m){
        assign(paste0('cox','.',party.name,'.',i), 
            coxph(Surv(time, Y) ~ . ,data=z1z2list[[1]][[i]],
            ties='breslow'), envir=.GlobalEnv)
    }
}

In [136]:
make.model(Z1.Z2.H1.100, 'H1')
make.model(Z1.Z2.H1.100, 'H2')
# 모델 생성

In [201]:
cox.H1.2[20]

$<NA>
NULL


In [210]:

cox.H1.2["coefficients"][1][2]

$<NA>
NULL


In [166]:
cox.H1.1

Call:
coxph(formula = Surv(time, Y) ~ ., data = z1z2list[[1]][[i]], 
    ties = "breslow")

         coef  exp(coef)   se(coef)      z       p
x1         NA         NA  0.0000000     NA      NA
x2  0.0185982  1.0187722  0.0074363  2.501 0.01238
x3 -0.1008344  0.9040827  0.0313294 -3.219 0.00129
x4 -0.1672417  0.8459951  0.0576417 -2.901 0.00371
x5 -3.1322843  0.0436180  0.9691840 -3.232 0.00123
x6  0.0004329  1.0004330  0.0096645  0.045 0.96427
x7  0.0070219  1.0070466  0.0043649  1.609 0.10767
x8  0.0148874  1.0149988  0.0195810  0.760 0.44708
x9  0.0027342  1.0027379  0.0045850  0.596 0.55095

Likelihood ratio test=74.85  on 8 df, p=5.28e-13
n= 370, number of events= 105 

In [146]:
total_model.H1 <- matrix(nrow=3, ncol=3)

total_model.H1[1,1] <- 1
total_model.H1

1,NA,NA
NA,NA,NA
NA,NA,NA


In [70]:
make.coefficient.matrix <- function(data.list, party.name)
{
    m <- data.list[1] %>% length
    p <- (data.list[[1]][1] %>% names %>% length) - 2
    
    # 각 데이터로부터의 행렬. m*2p 모양이다
    assign(paste0('total_model.',party.name), matrix(nrow=m, ncol=p)
    
    # 이 함수는 
    assign.by.char <- function(x, ...){
        eval.parent(assign(x, do.call(`[<-`, list(get(x), ...))))
    }
           
    for(i in 1:m){
        for(j in 1:p){
            
            assign(get(paste0('total_model.', party.name))[i,j] , get(paste0('cox.',party.name,'.',i))$coefficient[[j]])
#             a <- assign.by.char(paste0('total_model', party.name), i, j, get(paste0('cox.',party.name,'.',i))$coefficient[j])
#             assign(paste0('total_model',party.name), a)
            
            assign(get(paste0('total_model.', party.name))[i,j+p], get(paste0('cox.',party.name,'.',i))$coefficient[j,3])
            a <- assign.by.char(paste0('total_model', party.name), i, (j+P), summary(get(paste0('cox.',party.name,'.',i)))$coefficients[j,3])
            assign(paste0('total_model',party.name), a)
        }
    }
    assign(paste0('total_model',host), matrix(nrow=))
}
for(i in )

In [ ]:
# time vector
time.vector.function <- function(local.data)
{
    event<- select(filter(local.data, Y==1), time)
    grouped <- group_by(event, time)
    assign(paste0())
}
select(filter())

In [86]:
Z1.Z2.H1.100[[1]][1]

time,Y,x1,x2,x3,x4,x5,x6,x7,x8,x9
<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
25.0208,0,420,46,10,39.6,7.37,42,35,1.3,50
18.0208,0,420,62,9,32.2,7.31,40,32,0.3,70
4.2256,0,420,63,13,36.3,7.43,62,13,0.3,70
44.4562,0,420,24,7,36.0,7.24,44,11,0.6,60
8.4611,1,420,77,9,36.0,7.23,40,53,1.5,50
13.9409,0,420,67,4,36.2,7.50,26,48,6.3,60
2.0263,0,420,83,13,36.9,7.44,42,39,0.9,28
6.9861,0,420,78,13,37.8,7.41,44,63,0.2,100
5.5069,0,420,72,12,36.4,7.38,29,24,0.4,50


In [85]:
make.model(Z1.Z2.H1.100, H1)

ERROR: Error in Surv(time, Y): 객체 'Y'를 찾을 수 없습니다
